# Basic Example: Prompt+Model+OutputParser

- Author: [ChangJun Lee](https://www.linkedin.com/in/cjleeno1/)
- Design: []()
- Peer Review: []()
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab] [![Open in LangChain Academy]

## Overview

The most fundamental and commonly used case involves linking a prompt template with a model. To illustrate how this works, let us create a chain that asks for the capital cities of various countries.


### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Implementing the Comma-Separated List Output Parser](#implementing-the-comma-separated-list-output-parser)
- [Using Streamed Outputs](#using-streamed-outputs)

### References

- [LangChain ChatOpenAI API reference](https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html)
- [LangChain Core Output Parsers](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.list.CommaSeparatedListOutputParser.html#)
- [Python List Tutorial](https://docs.python.org/3.13/tutorial/datastructures.html)
---

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [ ]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [ ]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_openai",
        "langchain_community",
    ],
    verbose=False,
    upgrade=False,
)

In [ ]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "02-CommaSeparatedListOutputParser",
    }
)

---
You can alternatively set `OPENAI_API_KEY` in `.env` file and load it. 

[Note] This is not necessary if you've already set `OPENAI_API_KEY` in previous steps.

In [ ]:
# Configuration File for Managing API Key as an Environment Variable
from dotenv import load_dotenv

# Load API KEY Information
load_dotenv()

In [ ]:
# Set up LangSmith tracking at [https://smith.langchain.com](https://smith.langchain.com).
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# Enter the project name.
logging.langsmith("CH01-Basic")

Utilizing Prompt Templates

`PromptTemplate`

- A prompt template is used to create a complete prompt string by incorporating the user's input variables.
- Usage
  - `template`: A template string is a predefined format where curly braces '{}' are used to represent variables.

  - `input_variables`: The names of the variables to be inserted within the curly braces are defined as a list.

`input_variables`

- `input_variables` is a list that defines the names of the variables used in the `PromptTemplate`.

In [4]:
from langchain_teddynote.messages import stream_response  # Streaming Output
from langchain_core.prompts import PromptTemplate

The `from_template()` method is used to create a `PromptTemplate` object.

In [ ]:
# Define template
template = "What is the capital of {country}?"

# Create a `PromptTemplate` object using the `from_template` method.
prompt_template = PromptTemplate.from_template(template)
prompt_template

In [ ]:
# Generate the prompt.
prompt = prompt_template.format(country="Korea")
prompt

In [ ]:
# Generate the prompt.
prompt = prompt_template.format(country="USA")
prompt

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

## Chain Creation

### LCEL (LangChain Expression Language)

Here, we use LCEL to combine various components into a single chain.

![lcel.png](./images/lcel.png)

```
chain = prompt | model | output_parser
```

The `|` symbol works similarly to the [Unix pipe operator](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>), linking different components and passing the output of one component as the input to the next.

In this chain, user input is passed to the prompt template, and the output from the prompt template is then forwarded to the model. By examining each component individually, you can understand what happens at each step.

In [8]:
# Create the prompt as a `PromptTemplate` object.
prompt = PromptTemplate.from_template("Please explain {topic} in simple terms.")

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

chain = prompt | model

### Invoking `invoke()`

- Input values are provided in the form of a Python dictionary (key-value pairs).  
- When calling the `invoke()` function, these input values are passed as arguments.

In [9]:
# Set the topic in the `input` dictionary to 'The Principles of Learning in Artificial Intelligence Models'.
input = {"topic": "The Principles of Learning in Artificial Intelligence Models"}

In [ ]:
# Connect the `prompt` object and the `model` object using the pipe (`|`) operator.  
# Use the `invoke` method to pass the `input`.  
# This will return the message generated by the AI model.
chain.invoke(input)

Below is an example of outputting a streaming response:

In [ ]:
# Request for Streaming Output
answer = chain.stream(input)

# Streaming Output
stream_response(answer)

### Output Parser


In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

An output parser is added to the chain.

In [ ]:
# A processing chain is constructed by connecting the prompt, model, and output parser.
chain = prompt | model | output_parser

In [ ]:
# The `invoke` method of the `chain` object is used to pass the input.
chain.invoke(input)

In [ ]:
# Request for Streaming Output
answer = chain.stream(input)

# Streaming Output
stream_response(answer)

### Applying and Modifying Templates

- The prompt content below can be **modified** as needed for testing purposes.  
- The `model_name` can also be adjusted for testing.

In [ ]:
template = """
You are a seasoned English teacher with 10 years of experience. Please write an English conversation suitable for the given situation.  
Refer to the [FORMAT] for the structure.

#SITUATION:
{question}

#FORMAT:
- English Conversation:
- Translation(in Korean):
"""

# A prompt is generated using the prompt template.
prompt = PromptTemplate.from_template(template)

# Initialize the ChatOpenAI chat model.
model = ChatOpenAI(model_name="gpt-4o-mini")

# Initialize the string output parser.
output_parser = StrOutputParser()

In [ ]:
# Construct the chain.
chain = prompt | model | output_parser

In [ ]:
# Execute the completed Chain to obtain a response.
print(chain.invoke({"question": "I want to go to a restaurant and order food."}))

In [ ]:
# Execute the completed Chain to obtain a response  
# Request for Streaming Output
answer = chain.stream({"question": "ㅑ would like to go to a restaurant and order some food."})

# Streaming Output
stream_response(answer)

English Conversation:
- Hello, could I see the menu, please? 
- I'd like to order the grilled salmon and a side of mashed potatoes.
- Could I have a glass of water as well?
- Thank you!

Translation(in Korean):
- 안녕하세요, 메뉴판 좀 볼 수 있을까요?
- 구운 연어와 매시드 포테이토를 주문하고 싶어요.
- 물 한 잔도 주실 수 있나요?
- 감사합니다!

In [ ]:
# This time, set the question to 'Ordering Pizza in the US' and execute it.  
# Request for Streaming Output
answer = chain.stream({"question": "Ordering Pizza in the US"})

# Streaming Output
stream_response(answer)

English Conversation:
- Employee: "Hello, Tony's Pizza. How can I help you?"
- Customer: "Hi, I'd like to place an order for delivery, please."
- Employee: "Sure thing! What would you like to order?"
- Customer: "I'll have a large pepperoni pizza with extra cheese and a side of garlic bread."
- Employee: "Anything to drink?"
- Customer: "Yes, a 2-liter bottle of Coke, please."
- Employee: "Alright, your total comes to $22.50. Can I have your delivery address?"
- Customer: "It's 742 Evergreen Terrace."
- Employee: "Thank you. Your order will be there in about 30-45 minutes. Is there anything else I can help you with?"
- Customer: "No, that's everything. Thank you!"
- Employee: "Thank you for choosing Tony's Pizza. Have a great day!"

Translation(in Korean):
- 직원: "안녕하세요, 토니의 피자입니다. 어떻게 도와드릴까요?"
- 고객: "안녕하세요, 배달 주문하고 싶은데요."
- 직원: "네, 무엇을 주문하시겠어요?"
- 고객: "큰 사이즈의 페퍼로니 피자에 치즈 추가하고, 마늘빵 하나 주세요."
- 직원: "음료는 드릴까요?"
- 고객: "네, 콜라 2리터 한 병 주세요."
- 직원: "알겠습니다, 합계는 $22.50입니다. 배달 주소를 알려주시겠어요?"
- 고객: "742 에버그린 테라스입니다."
- 직원: "감사합니다. 주문하신 음식은 대략 30-45분 내에 도착할 예정입니다. 다른 도움이 필요하신가요?"
- 고객: "아니요, 이게 다예요. 감사합니다!"
- 직원: "토니의 피자를 선택해주셔서 감사합니다. 좋은 하루 되세요!"
